## Quicktour

参数高效微调（PEFT Parameter-Efficient Fine-Tuning​​）为大型预训练模型提供了参数高效的微调方法。传统做法是针对每个下游任务对模型的所有参数进行微调，但由于如今模型的参数数量庞大，这种方法的成本越来越高，也越来越不切实际。相反，训练较少的提示参数或使用诸如低秩适应（LoRA low-rank adaptation）之类的重新参数化方法来减少可训练参数的数量会更高效。

本快速入门将向您展示 PEFT 的主要功能，以及如何在普通消费者设备上训练或运行通常无法触及的大型模型的推理。

### Train

每种参数高效微调（PEFT）方法都由一个 PeftConfig 类定义，该类存储构建PeftModel所需的所有重要参数。例如，若要使用低秩适应（LoRA）进行训练，请加载并创建一个 LoraConfig 类，并指定以下参数：
- task_type: 目标任务
- inference_mode：是否使用模型来进行预测
- r: 低秩矩阵的维度
- lora_alpha：低秩矩阵的缩放因子
- lora_dropout：LoRA 层的丢弃概率

In [9]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

一旦设置好 LoraConfig，就可以使用 get_peft_model() 函数创建一个 PeftModel。该函数需要一个基础模型（可以从 Transformers 库中加载）以及包含用于使用 LoRA 训练模型的参数的 LoraConfig。

加载您想要微调的基础模型。

In [10]:
from transformers import AutoModelForSeq2SeqLM

checkpoint = "bigscience/mt0-large"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

使用 `get_peft_model()` 函数`model`和 `peft_config` 包装起来以创建一个 `PeftModel`。要了解模型中可训练参数的数量，请使用 `print_trainable_parameters` 方法。

In [11]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


在 bigscience/mt0-large 的 12 亿参数中，您只需训练其中的 0.19%！

就是这样 🎉！现在您可以使用 Transformers Trainer、Accelerate 或任何自定义的 PyTorch 训练循环来训练模型。

例如，若要使用 Trainer 类进行训练，请设置一个带有某些训练超参数的 TrainingArguments 类。

In [ ]:
training_args = TrainingArguments(
    output_dir="your-name/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

## PEFT configuration and model

对于任何参数高效微调（PEFT）方法，您都需要创建一个config，其中包含所有指定应如何应用该 PEFT 方法的参数。配置设置完成后，将其与base model一起传递给 get_peft_model() 函数，以创建可训练的 PeftModel。

**prompt tuning**

提示调优将所有任务都视为生成任务，并在输入中添加特定于任务的提示，该提示会独立更新。`prompt_tuning_init_text` 参数指定了如何微调模型（在本例中，是判断推文是否为投诉）。为了获得最佳效果，`prompt_tuning_init_text` 应该具有与应预测的`token`数量相同的`token`数。为此，您可以将 `num_virtual_tokens` 设置为 `prompt_tuning_init_text` 的标记数。


创建一个`PromptTuningConfig`，其中包含任务类型、用于训练模型的初始提示调优文本、要添加和学习的虚拟`token`数量，以及一个`tokenizer`。

In [ ]:
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model

prompt_tuning_init_text = "Classify if the tweet is a complaint or no complaint.\n"
peft_config = PromptTuningConfig(
    task_type="CAUSAL_LM",
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=len(tokenizer(prompt_tuning_init_text)["input_ids"]),
    prompt_tuning_init_text=prompt_tuning_init_text,
    tokenizer_name_or_path="bigscience/bloomz-560m",
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

**Prefix tuning**

前缀调优在模型的所有层中添加特定任务的参数，这些参数由一个单独的前馈网络进行优化。使用任务类型和要添加及学习的虚拟token数量创建一个 PrefixTuningConfig 配置。

In [ ]:
from peft import PrefixTuningConfig, get_peft_model

peft_config = PrefixTuningConfig(task_type="CAUSAL_LM", num_virtual_tokens=20)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"trainable params: 983,040 || all params: 560,197,632 || trainable%: 0.1754809274167014"

**p-tuning**

P 调优添加了一个可训练的嵌入张量，提示token可以在输入序列中的任何位置添加。使用任务类型、要添加和学习的虚拟token数量以及用于学习提示参数的编码器隐藏大小创建一个 PromptEncoderConfig。

In [ ]:
from peft import PromptEncoderConfig, get_peft_model

peft_config = PromptEncoderConfig(
    task_type="CAUSAL_LM", 
    num_virtual_tokens=20, 
    encoder_hidden_size=128
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()